In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

# Load CSV
df = pd.read_csv('short_stories.csv')
df

,Id,Title,Author,Rating,Img,Img_description,Year,Words,Minutes,Description,Genres,Blurb,PDF,Epub,Read
0,0,"The £1,000,000 Bank-Note",Mark Twain,3.0,https://libraryofshortstories.s3.ap-southeast-...,"A beared man in a suit, hat, and tie. Browsing...",1893.0,8226,33,"A man is given a bank note worth £1,000,000 as...",Quirky,"‘Give me the change, please.’",https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
1,1,"The $25,000 Jaw",Richard Connell,3.0,https://libraryofshortstories.s3.ap-southeast-...,A clenched fist. Graphite filter.,1922.0,6592,27,Croly Addicks is chinless both metaphorically ...,Quirky,"“Aw, wadda yuh expeck of Chinless?” returned t...",https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
2,2,2BR02B,Kurt Vonnegut,4.0,https://libraryofshortstories.s3.ap-southeast-...,"A solitary,colourless rotary telephone resting...",1962.0,2519,11,"In a future where death has been solved, and t...",Sci-fi;Tragedy,The room was being redecorated. It was being r...,https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
3,3,39,Henry Lawson,3.0,https://libraryofshortstories.s3.ap-southeast-...,"A man in a tux, tipping his hat. Graphite Filter.",1906.0,254,2,The author looks forward to living in his fort...,Poetry,I only woke this morning - To find the world i...,https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
4,4,About Barbers,Mark Twain,3.0,https://libraryofshortstories.s3.ap-southeast-...,"Half of a stern, staring face. Graphite filter",1871.0,1576,7,A man awkwardly does his best to avoid falling...,Quirky,About this time I was amusing myself trying to...,https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,839,Xeethra,Clark Ashton Smith,5.0,https://libraryofshortstories.s3.ap-southeast-...,Drawing of a young boy in front of a strange t...,1934.0,7394,30,A young goat shepherd discovers a mysterious c...,Fantasy;Adventure,"“These things were illusions, designed to lead...",https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
840,840,The Yellow Face,Arthur Conan Doyle,NaN,https://libraryofshortstories.s3.ap-southeast-...,A man in a bowler hat and a popped collar. Gra...,1893.0,7497,30,"Grand Munro comes to Holmes, suspicious of his...",Mystery;Adventure,“One does not like to speak of one’s domestic ...,https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
841,841,The Yellow Wallpaper,Charlotte Perkins Gilman,4.0,https://libraryofshortstories.s3.ap-southeast-...,Drawing of a woman in a 19th century dress wri...,1892.0,6089,25,"A woman, restrained to her house by her husban...",Gothic;Tragedy;Literary,"At first he meant to repaper the room, but aft...",https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...
842,842,Young Goodman Brown,Nathaniel Hawthorne,NaN,https://libraryofshortstories.s3.ap-southeast-...,A large twisting tree in a forest. Graphite fi...,1835.0,5229,21,A man leaves his wife to head off into the for...,Literary;Gothic,But the only thing about him that could be fix...,https://www.libraryofshortstories.com/storiesp...,https://www.libraryofshortstories.com/storiese...,https://www.libraryofshortstories.com/onlinere...


In [23]:
# Create output folder
os.makedirs('story_texts', exist_ok=True)

# Base URL for relative paths
base_url = 'https://www.libraryofshortstories.com'

# Loop through each row
for index, row in df.iterrows():
    read_url = row['Read']
    story_id = str(row['Id'])  # Ensure it's a string
    print(f"Processing ID {story_id}: {read_url}")

    try:
        # Get the HTML page
        response = requests.get(read_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all <link rel="preload"> tags
        preload_links = soup.find_all('link', rel='preload')
        txt_links = [link['href'] for link in preload_links if link.get('href', '').endswith('.txt')]

        if txt_links:
            story_path = txt_links[0]
            story_url = urljoin(base_url, story_path)

            # Download the .txt file
            story_response = requests.get(story_url)
            story_response.raise_for_status()

            # Save using the ID as filename
            save_path = os.path.join('story_texts', f'{story_id}.txt')
            with open(save_path, 'wb') as f:
                f.write(story_response.content)

            print(f"[✓] Saved: {save_path}")
        else:
            print(f"[!] No .txt file found at: {read_url}")

    except Exception as e:
        print(f"[X] Error processing {read_url}: {e}")

Processing ID 0: https://www.libraryofshortstories.com/onlinereader/the-1000000-bank-note
[✓] Saved: story_texts/0.txt
Processing ID 1: https://www.libraryofshortstories.com/onlinereader/the-25000-jaw
[✓] Saved: story_texts/1.txt
Processing ID 2: https://www.libraryofshortstories.com/onlinereader/2br02b
[✓] Saved: story_texts/2.txt
Processing ID 3: https://www.libraryofshortstories.com/onlinereader/39
[✓] Saved: story_texts/3.txt
Processing ID 4: https://www.libraryofshortstories.com/onlinereader/about-barbers
[✓] Saved: story_texts/4.txt
Processing ID 5: https://www.libraryofshortstories.com/onlinereader/about-love
[✓] Saved: story_texts/5.txt
Processing ID 6: https://www.libraryofshortstories.com/onlinereader/according-to-their-lights
[✓] Saved: story_texts/6.txt
Processing ID 7: https://www.libraryofshortstories.com/onlinereader/across-the-straits
[✓] Saved: story_texts/7.txt
Processing ID 8: https://www.libraryofshortstories.com/onlinereader/an-adventure-in-futurity
[✓] Saved: stor